### Dependencies and imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install -U sentence-transformers
!pip install -U transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.9 MB/s 
     |████████████████████████████████| 182 kB 90.2 MB/s 
     |████████████████████████████████| 7.6 MB 57.2 MB/s 


In [ ]:
import os
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader
from torchvision import datasets, transforms
# from sentence_transformers import SentenceTransformer
from transformers import BertTokenizer, BertModel

import pandas as pd
import numpy as np
import tqdm, glob

from torch.utils.data import Dataset, DataLoader
from torchvision.models.feature_extraction import create_feature_extractor

import random

In [ ]:
%cd '/content/drive/MyDrive/Deep Learning Project/multi-modal-music-genre-classification/'
from CNN import CNN
from utils import load_data
from TransformerEncoder import TransformerEncoder

/content/drive/.shortcut-targets-by-id/171ooq7ooYWU0qxPszBUOL0gPzYTH3uuB/Deep Learning Project/multi-modal-music-genre-classification


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


### Lyric Module

In [ ]:
class LyricModule(nn.Module):
  def __init__(self, pretrained_lyric_model): #, max_seq_len):
    super(LyricModule, self).__init__()

    # # Pretrained lyric model to get lyric embeddings
    self.pretrained = pretrained_lyric_model
    # print("Max Sequence Length:", self.pretrained.max_seq_length)
    # self.pretrained.max_seq_length = max_seq_len # common for BERT and its derivatives
    # print("Max Sequence Length:", self.pretrained.max_seq_length)

    # self.lyric_dim = self.pretrained.get_sentence_embedding_dimension()
    self.embedding_dim = 768

  def forward(self, input_ids, attention_mask):
    '''
    input_ids: shape (batch_size, num_tokens)
    attention_mask: shape (batch_size, num_tokens)

    embeddings: shape (batch_size, num_tokens, embedding_dim)
    pooled: shape (batch_size, embedding_dim)
    '''
    # embeddings = self.pretrained.encode(lyrics, convert_to_tensor=True).to(device)
    bert_output = self.pretrained(input_ids = input_ids, attention_mask = attention_mask)
    embeddings = bert_output.last_hidden_state
    pooled = bert_output.pooler_output

    return embeddings, pooled

### CMA Module

In [ ]:
class CMAModule(nn.Module):
  def __init__(self, hyp_params):
    super(CMAModule, self).__init__()

    self.orig_d_l, self.orig_d_a = hyp_params['orig_d_l'], hyp_params['orig_d_a']
    self.d_l, self.d_a = 32, 32
    self.num_heads = hyp_params['num_heads']
    self.layers = hyp_params['layers']
    self.attn_dropout = hyp_params['attn_dropout']
    self.attn_dropout_a = hyp_params['attn_dropout_a']
    self.relu_dropout = hyp_params['relu_dropout']
    self.res_dropout = hyp_params['res_dropout']
    self.out_dropout = hyp_params['out_dropout']
    self.embed_dropout = hyp_params['embed_dropout']
    self.attn_mask = hyp_params['attn_mask']

    combined_dim = self.d_l + self.d_a
    output_dim = hyp_params['output_dim']

    # Init the Layers
    # 1.  Temporal Convolutional Layers
    self.proj_lyrics = nn.Conv1d(self.orig_d_l, self.d_l, kernel_size=1, padding=0, bias=False)
    self.proj_audio = nn.Conv1d(self.orig_d_a, self.d_a, kernel_size=1, padding=0, bias=False)

    # 2. CrossModal Attentions
    self.trans_l_with_a = self.get_network(self_type='la')
    self.trans_a_with_l = self.get_network(self_type='al')
    
    # 3.  Self Attentions
    self.trans_l_self = self.get_network(self_type='l_mem', layers=3)
    self.trans_a_self = self.get_network(self_type='a_mem', layers=3)

    # Projection layers???
    self.proj1 = nn.Linear(combined_dim, combined_dim)
    self.proj2 = nn.Linear(combined_dim, combined_dim)
    self.out_layer = nn.Linear(combined_dim, output_dim)
    

  def get_network(self, self_type='l', layers=-1):
      if self_type in ['l', 'al', 'vl']:
          embed_dim, attn_dropout = self.d_l, self.attn_dropout
      elif self_type in ['a', 'la', 'va']:
          embed_dim, attn_dropout = self.d_a, self.attn_dropout_a
      elif self_type == 'l_mem':
          embed_dim, attn_dropout = 1*self.d_l, self.attn_dropout
      elif self_type == 'a_mem':
          embed_dim, attn_dropout = 1*self.d_a, self.attn_dropout
      else:
          raise ValueError("Unknown network type")
      
      return TransformerEncoder(embed_dim=embed_dim,
                                num_heads=self.num_heads,
                                layers=max(self.layers, layers),
                                attn_dropout=attn_dropout,
                                relu_dropout=self.relu_dropout,
                                res_dropout=self.res_dropout,
                                embed_dropout=self.embed_dropout,
                                attn_mask=self.attn_mask)
      
    

  def forward(self, x_l, x_a):
    # x_l : lyrics
    # x_a : audio

    # Using 1D Convolution Layer
    x_l = x_l.transpose(1, 2)
    x_a = x_a.transpose(1, 2)
    proj_x_l = self.proj_lyrics(x_l)
    proj_x_a = self.proj_audio(x_a)
    proj_x_l = proj_x_l.permute(2,0,1)
    proj_x_a = proj_x_a.permute(2,0,1)

    # Cross Modal Attention
    h_l_with_as = self.trans_l_with_a(proj_x_l, proj_x_a, proj_x_a)
    # h_ls = self.trans_l_self(h_l_with_as) # Self Attention
    # last_h_l = last_hs = h_ls[-1]
    last_h_l = h_l_with_as[-1]  # Trying without Self Attention Layers
    # last_l_with_as = torch.flatten(h_l_with_as.permute(1,0,2), start_dim=1)  # 64, 512 * 32

    h_a_with_ls = self.trans_a_with_l(proj_x_a, proj_x_l, proj_x_l)
    # h_as = self.trans_a_self(h_a_with_ls) # Self Attention
    # last_h_a = last_hs = h_as[-1]
    last_h_a = h_a_with_ls[-1]   # Trying without Self Attention Layers
    # last_a_with_ls = torch.flatten(h_a_with_ls.permute(1,0,2), start_dim=1)  # 64, w*h * 32

    last_hs = torch.cat([last_h_l, last_h_a], dim=1)  # Concat both LA and AL
    # last_hs = torch.cat([last_l_with_as, last_a_with_ls], dim=1)  # Concat both LA and AL

    # Trying only L with A
    # last_hs = last_l_with_as

    last_hs_proj = self.proj2(F.dropout(F.relu(self.proj1(last_hs)), p=self.out_dropout, training=self.training))

    last_hs_proj += last_hs
      
    output = self.out_layer(last_hs_proj)

    return output


In [ ]:
cma_hyp_params = {
    'orig_d_l': 768,  # dimension_lyrics
    'orig_d_a': 96,  # dimension_audio
    'num_heads': 8,   # What is this?
    'layers': 3,
    'attn_dropout': 0.2,
    'attn_dropout_a': 0.2,
    'relu_dropout': 0.2,
    'res_dropout': 0.2,
    'out_dropout': 0.2,
    'embed_dropout': 0.2,
    'attn_mask': False,
    'output_dim': 5
}

### Genre Classifier Module

In [ ]:
class GenreClassifier(nn.Module):
    def __init__(
        self,
        spectrogram_model,
        lyric_model,
        cma_model,
        num_classes
        ):
        super(GenreClassifier, self).__init__()

        self.lyric_model = lyric_model
        self.spectrogram_model = spectrogram_model
        self.cma_model = CMAModule(cma_hyp_params)

        self.num_classes = num_classes

        self.avgpool = nn.AvgPool2d(2, stride=2)
        self.conv4 = nn.Conv2d(in_channels=48, out_channels=96, kernel_size=5, stride =1, padding = 'same')
        self.conv5 = nn.Conv2d(in_channels=96, out_channels=96, kernel_size=5, stride =1, padding = 'same')

        # FC layers after CMA
        # self.linear = nn.Linear(self.cma_model.embedding_size, self.num_classes)

        # self.lyric_linear_layers = nn.Sequential(
        #     nn.Linear(self.lyric_model.embedding_dim, 512),
        #     nn.ReLU(),
        #     nn.Linear(512, 128),
        #     nn.ReLU(),
        #     nn.Linear(128, 64),
        #     nn.ReLU(),
        #     # nn.Linear(64, self.num_classes)
        # )
        # self.cnn_linear_layers = nn.Sequential(
        #     nn.ReLU(),
        #     # nn.Linear(64, self.num_classes)
        # )
        # self.concat_linear_layers = nn.Sequential(
        #     nn.Linear(64+64, self.num_classes)
        # )

    def forward(self, input_ids, attention_mask, spectrograms):
      '''
      input_ids: shape (batch_size, num_tokens)
      attention_mask: shape (batch_size, num_tokens)
      spectrograms: shape (batch_size, ??)
      '''
      #### get embeddings 
      token_embeddings, pooled = self.lyric_model(input_ids, attention_mask)
      spectrogram_embeddings_raw = self.spectrogram_model(spectrograms.float())

      # spectrogram_embeddings_3d = spectrogram_embeddings[:, None, :]

      spectrogram_embeddings = spectrogram_embeddings_raw['linear1']
      spectrogram_embeddings_conv = spectrogram_embeddings_raw['conv3']

      spectrogram_embeddings_conv = nn.functional.relu(self.avgpool(self.conv4(spectrogram_embeddings_conv)))
      spectrogram_embeddings_conv = nn.functional.relu(self.avgpool(self.conv5(spectrogram_embeddings_conv)))

      spectrogram_embeddings_conv = torch.flatten(spectrogram_embeddings_conv, start_dim=2)

      #### linear layers      

      # print(spectrogram_embeddings_conv3.shape)
      # print(token_embeddings.shape)

      # output = torch.cat((self.cnn_linear_layers(spectrogram_embeddings), self.lyric_linear_layers(pooled)), 1)
      # output = self.concat_linear_layers(output)
      # print(output.shape)
      
      spectrogram_embeddings_conv = spectrogram_embeddings_conv.permute(0, 2, 1)
      output_cma = self.cma_model(token_embeddings, spectrogram_embeddings_conv)
      # print(output_cma.shape)

      return output_cma

      #TODO: add linear layers 
      # output = self.linear(output)

      # return output

## Training

### Load data

In [ ]:
BATCH_SIZE = 64

In [ ]:
# pretrained_lyric_model = SentenceTransformer('all-mpnet-base-v2')
# pretrained_lyric_model.get_sentence_embedding_dimension()
pretrained_lyric_model = BertModel.from_pretrained('bert-base-uncased')
pretrained_lyric_model.eval()
# FREEZE BERT MODEL
for param in pretrained_lyric_model.parameters():
  param.requires_grad = False

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
genre_list = np.load("genre_list.npy")
# zipped_data, genre_list = load_data()
NUM_CLASSES = len(genre_list)
print(NUM_CLASSES)

5


In [ ]:
# custom data loader

class CustomDataset(Dataset):
    def __init__(self,split):
        print("Creating custom dataset ...")

        self.specs_path = f"{split}_specs.npy"
        self.tokens_path = f"{split}_tokens.npy"
        self.attention_mask_path = f"{split}_attention_masks.npy"
        self.y_path = f"{split}_y.npy"


        # # self.input, self.target = zip(*split)
        # # self.melspec, self.lyrics = zip(*self.input)

        self.melspec = torch.unsqueeze(torch.tensor(np.stack(np.load(self.specs_path, allow_pickle=True))), 1)
        self.tokens = torch.tensor(np.load(self.tokens_path))
        self.attention_mask = torch.tensor(np.load(self.attention_mask_path))
        self.target = torch.tensor(np.load(self.y_path))
        # #### reshape spectrograms
        # self.melspec = torch.from_numpy(np.array([x.reshape( (1, 128, 431) ) for x in self.melspec]))
        print("melspec shape:", self.melspec.shape)

        # #### tokenize lyrics
        # # self.lyrics = list(self.lyrics)
        # # tokenizer_output = tokenizer(self.lyrics, padding=True, truncation=True, return_tensors="pt")
        # # self.input_ids = tokenizer_output['input_ids']
        # # self.attention_mask = tokenizer_output['attention_mask']
        print("tokens shape:", self.tokens.shape)
        print("attention mask shape:", self.attention_mask.shape)
        print("target shape:", self.target.shape)

        # self.target = torch.tensor(target)
        # print("-------------------")


        
    def __getitem__(self, index):
        
        # #melspec, lyric data-pair

        data_pair = (self.melspec[index], (self.tokens[index], self.attention_mask[index]))
        label = self.target[index]
        return data_pair, label



    def __len__(self):
        count = len(self.target)
        return count 

In [ ]:
#### data separation 
# dataset = zipped_data
# random.shuffle(dataset)
# del(dataset)

# trainDataset = CustomDataset(train, tokenizer, pretrained_lyric_model)
# del(train)
# valDataset = CustomDataset(val, tokenizer, pretrained_lyric_model)
# del(val)
# testDataset = CustomDataset(test, tokenizer, pretrained_lyric_model)
# del(test)

# del(pretrained_lyric_model)
# del(tokenizer)
trainDataset = CustomDataset("train")
valDataset = CustomDataset("val")
testDataset = CustomDataset("test")
# del(specs)
# del(tokens)
# del(attention_mask)

trainLoader = DataLoader(trainDataset, batch_size=BATCH_SIZE)
valLoader = DataLoader(valDataset, batch_size=BATCH_SIZE)
testLoader = DataLoader(testDataset)


Creating custom dataset ...
melspec shape: torch.Size([6896, 1, 128, 431])
tokens shape: torch.Size([6896, 512])
attention mask shape: torch.Size([6896, 512])
target shape: torch.Size([6896])
Creating custom dataset ...
melspec shape: torch.Size([862, 1, 128, 431])
tokens shape: torch.Size([862, 512])
attention mask shape: torch.Size([862, 512])
target shape: torch.Size([862])
Creating custom dataset ...
melspec shape: torch.Size([863, 1, 128, 431])
tokens shape: torch.Size([863, 512])
attention mask shape: torch.Size([863, 512])
target shape: torch.Size([863])


In [ ]:
# del(trainDataset)
# del(valDataset)
del(testDataset)
# del(zipped_data)
# del(trainLoader)
# del(testLoader)
# del(valLoader)
# del(model)

In [ ]:
import gc
gc.collect()

508

### Define model

In [ ]:
pretrained_cnn_model = CNN(num_classes = 5).to(device)
pretrained_cnn_model.load_state_dict(torch.load('CNN_pytorch_training/saved_model.pth'))

<All keys matched successfully>

In [ ]:
lyric_model = LyricModule(pretrained_lyric_model)

spectrogram_model = create_feature_extractor(pretrained_cnn_model, return_nodes=['linear1', 'conv3'])

# FREEZE CNN MODEL
# for param in spectrogram_model.parameters():
#   param.requires_grad = False

cma_model = None

model = GenreClassifier(spectrogram_model, lyric_model, cma_model, 5).float().to(device)

In [ ]:
for param in model.named_parameters():
  print(param[0], param[1][1].requires_grad)

lyric_model.pretrained.embeddings.word_embeddings.weight False
lyric_model.pretrained.embeddings.position_embeddings.weight False
lyric_model.pretrained.embeddings.token_type_embeddings.weight False
lyric_model.pretrained.embeddings.LayerNorm.weight False
lyric_model.pretrained.embeddings.LayerNorm.bias False
lyric_model.pretrained.encoder.layer.0.attention.self.query.weight False
lyric_model.pretrained.encoder.layer.0.attention.self.query.bias False
lyric_model.pretrained.encoder.layer.0.attention.self.key.weight False
lyric_model.pretrained.encoder.layer.0.attention.self.key.bias False
lyric_model.pretrained.encoder.layer.0.attention.self.value.weight False
lyric_model.pretrained.encoder.layer.0.attention.self.value.bias False
lyric_model.pretrained.encoder.layer.0.attention.output.dense.weight False
lyric_model.pretrained.encoder.layer.0.attention.output.dense.bias False
lyric_model.pretrained.encoder.layer.0.attention.output.LayerNorm.weight False
lyric_model.pretrained.encoder.lay

### Hyperparameters

In [ ]:
learning_rate = 1e-4
epochs = 5

### Loss function

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

### Training Loop

In [ ]:
import tqdm

In [ ]:
# iterate through all the epochs
min_valid_loss = 20
for epoch in range(3):
    # go through all the batches generated by dataloader

    ############## train loop
    train_loss = 0.0
    for i, (X, y) in tqdm.notebook.tqdm(enumerate(trainLoader)):

      specs = X[0]
      input_ids, attention_mask = X[1]
      pred = model(input_ids.to(device), attention_mask.to(device), specs.to(device))

      optimizer.zero_grad()
      # calculate loss
      loss = criterion(pred, y.to(device))
      # credit assignment
      loss.backward()
      # update model weights
      optimizer.step()

      train_loss += loss.item()
    
    ############## validation loop
    model.eval()
    valid_loss = 0.0 
    for i, (X, y) in tqdm.notebook.tqdm(enumerate(valLoader)):

      specs = X[0]
      input_ids, attention_mask = X[1]
      pred = model(input_ids.to(device), attention_mask.to(device), specs.to(device))

      # calculate loss
      loss = criterion(pred, y.to(device))
      valid_loss += loss.item() 
    

    print(f'Epoch {epoch+1} \t\t Training Loss: {train_loss / len(trainLoader)} \t\t Validation Loss: {valid_loss / len(valLoader)}')
    
    if min_valid_loss > valid_loss:
        print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f}) \t Saving The Model')
        min_valid_loss = valid_loss
        
        # Saving State Dict
        # torch.save(model.state_dict(), '/content/drive/MyDrive/multi-modal-music-genre-classification/CNN_pytorch_training/saved_model.pth')
        torch.save(model.state_dict(), 'model_cma_8ECMATrainingRename.pth')

0it [00:00, ?it/s]

0it [00:00, ?it/s]

Epoch 1 		 Training Loss: 1.3101735600718745 		 Validation Loss: 1.1087404957839422
Validation Loss Decreased(20.000000--->15.522367) 	 Saving The Model


0it [00:00, ?it/s]

0it [00:00, ?it/s]

Epoch 2 		 Training Loss: 1.0238980899254482 		 Validation Loss: 1.05045571071761
Validation Loss Decreased(15.522367--->14.706380) 	 Saving The Model


0it [00:00, ?it/s]

0it [00:00, ?it/s]

Epoch 3 		 Training Loss: 0.8752337258171152 		 Validation Loss: 1.046847756419863
Validation Loss Decreased(14.706380--->14.655869) 	 Saving The Model


In [ ]:
model = GenreClassifier(spectrogram_model, lyric_model, cma_model, 5).float().to(device)
model.load_state_dict(torch.load("model_cma_10E60A5ConvLayers.pth"))

<All keys matched successfully>

In [ ]:
# model = GenreClassifier(spectrogram_model, lyric_model, cma_model, NUM_CLASSES).float().to(device)
# model.load_state_dict(torch.load("model_cma_8ECMATrainingRename.pth"))
############## test loop
model.eval()
# model.train()
test_loss = 0.0
total = 0.0
num_correct = 0.0

# loop over all batches in test set
for i, (X, y) in tqdm.notebook.tqdm(enumerate(testLoader)):

  specs = X[0]
  input_ids, attention_mask = X[1]
  pred = model(input_ids.to(device), attention_mask.to(device), specs.to(device))

  # pred = pretrained_cnn_model(specs.float())
  res = torch.argmax(pred, 1)

  targets = y.to(device)
  num_correct += (res.item() == targets.item())
  total += targets.size(0)
  test_loss += criterion(pred, targets).item()
    

print(f"Test Accuracy of the model: {float(num_correct)/float(total)*100:.2f}")

0it [00:00, ?it/s]

Test Accuracy of the model: 60.25


In [ ]:
%pwd

'/content/drive/.shortcut-targets-by-id/199GHGZBtQOuthTQ8c7i5neHpvqm7epIw/multi-modal-music-genre-classification'

In [ ]:
torch.save(model.state_dict(), "model_cma_5E56AFlattenInstead.pth")

In [ ]:
import sys
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
                         key= lambda x: -x[1]):
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

                   zipped_data: 72.1 KiB
                         train: 53.9 KiB
                          test:  6.8 KiB
                           val:  6.8 KiB
                 BertTokenizer:  2.0 KiB
                           _i6:  1.8 KiB
                          _iii:  1.6 KiB
                           _i9:  1.6 KiB
                     BertModel:  1.4 KiB
                           CNN:  1.4 KiB
                    DataLoader:  1.0 KiB
                       Dataset:  1.0 KiB
               GenreClassifier:  1.0 KiB
                 CustomDataset:  1.0 KiB
                           _ii:  598.0 B
                          _i10:  598.0 B
                          _i12:  582.0 B
                            _i:  533.0 B
                          _i11:  533.0 B
                           _i3:  510.0 B
                           _oh:  248.0 B
                           Out:  248.0 B
                           _ih:  200.0 B
                            In:  200.0 B
                